In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch





In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
from datasets import load_dataset
train_data=load_dataset("wmt16","de-en", split="train[:50000]")
val_data=load_dataset("wmt16","de-en", split="validation")
test_data=load_dataset("wmt16","de-en", split="test")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


In [4]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [5]:
source_lang = "de"
target_lang = "en"
prefix = "translate German to English: "

In [6]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

In [7]:
tokenized_train_data = train_data.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

In [8]:
tokenized_val_data = val_data.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
tokenized_test_data = test_data.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-03-31 08:17:26.461628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 08:17:26.461730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 08:17:26.735689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [18]:
import evaluate

metric = evaluate.load("sacrebleu")
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [17]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 908.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.6 MB/s eta 0:00:00 0:00:01


In [19]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
for example in tokenized_train_data:
    example['input_ids'] = torch.tensor(example['input_ids']).to(device)
    example['attention_mask'] = torch.tensor(example['attention_mask']).to(device)
    example['labels'] = torch.tensor(example['labels']).to(device)

for example in tokenized_val_data:
    example['input_ids'] = torch.tensor(example['input_ids']).to(device)
    example['attention_mask'] = torch.tensor(example['attention_mask']).to(device)
    example['labels'] = torch.tensor(example['labels']).to(device)

In [21]:
for param in model.decoder.block[-1].parameters():
    param.requires_grad = True

In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir="translation_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
  
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    
    push_to_hub=False,
)

from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_train_data, batch_size=training_args.per_device_train_batch_size, shuffle=True)

val_dataloader = DataLoader(tokenized_val_data, batch_size=training_args.per_device_eval_batch_size)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader.dataset,
    eval_dataset=val_dataloader.dataset,


    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.442400,2.140527,7.901700,16.933600
2,2.329500,2.099344,8.483000,16.926200
3,2.267600,2.077590,8.704000,16.922500
4,2.235700,2.066865,8.862900,16.941000
5,2.223800,2.063809,8.950300,16.938200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. 

TrainOutput(global_step=7815, training_loss=2.332265104853947, metrics={'train_runtime': 2577.5927, 'train_samples_per_second': 96.99, 'train_steps_per_second': 3.032, 'total_flos': 8051445704687616.0, 'train_loss': 2.332265104853947, 'epoch': 5.0})

In [23]:
trainer.save_model("translation_model")
import pickle


with open('/kaggle/working/translation_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [24]:
from transformers import pipeline

translator = pipeline("translation", model="translation_model",device=device)



from tqdm import tqdm



def generate_translations(data):
    translations = []
    for example in tqdm(data, desc="Translating"):
        input_text = "translate German to English: " + example["translation"]["de"]
        translated_text = translator(input_text)[0]['translation_text']
        translations.append(translated_text)
    return translations

test_translations = generate_translations(test_data)



/opt/conda/lib/python3.10/site-packages/transformers/pipelines/__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(
Translating:   0%|          | 10/2999 [00:02<13:31,  3.68it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Translating: 100%|██████████| 2999/2999 [15:31<00:00,  3.22it/s]


In [25]:
test_refs = [example["translation"]["en"] for example in test_data]
#calculate bleu score

import nltk
from nltk.translate.bleu_score import corpus_bleu
def calculate_bleu_scores(actual_sentences, translated_sentences):
    bleu_1 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(1, 0, 0, 0))
    bleu_2 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 1, 0, 0))
    bleu_3 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 1, 0))
    bleu_4 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 0, 1))
    return bleu_1, bleu_2, bleu_3, bleu_4




#validation_bleu_1, validation_bleu_2, validation_bleu_3, validation_bleu_4 = calculate_bleu_scores(validation_refs, validation_translations)
test_bleu_1, test_bleu_2, test_bleu_3, test_bleu_4 = calculate_bleu_scores(test_refs, test_translations)


print("\nTest BLEU-1 Score:", test_bleu_1)
print("Test BLEU-2 Score:", test_bleu_2)
print("Test BLEU-3 Score:", test_bleu_3)
print("Test BLEU-4 Score:", test_bleu_4)



Test BLEU-1 Score: 0.4482890188801773
Test BLEU-2 Score: 0.21180991320789933
Test BLEU-3 Score: 0.1108737333816971
Test BLEU-4 Score: 0.061670159513581305


In [26]:
import evaluate

bleu = evaluate.load("bleu")
results_1 = bleu.compute(predictions=test_translations, references=test_refs, max_order=1)
print(results_1)
results_2 = bleu.compute(predictions=test_translations, references=test_refs, max_order=2)
print(results_2)
results_3 = bleu.compute(predictions=test_translations, references=test_refs, max_order=3)
print(results_3)
results_4 = bleu.compute(predictions=test_translations, references=test_refs, max_order=4)
print(results_4)

{'bleu': 0.5262450627132879, 'precisions': [0.5315606244388085], 'brevity_penalty': 0.9900000837512514, 'length_ratio': 0.9900497512437811, 'translation_length': 63481, 'reference_length': 64119}
{'bleu': 0.36582935681843914, 'precisions': [0.5315606244388085, 0.2568820991022271], 'brevity_penalty': 0.9900000837512514, 'length_ratio': 0.9900497512437811, 'translation_length': 63481, 'reference_length': 64119}
{'bleu': 0.26621435971067786, 'precisions': [0.5315606244388085, 0.2568820991022271, 0.142397411588707], 'brevity_penalty': 0.9900000837512514, 'length_ratio': 0.9900497512437811, 'translation_length': 63481, 'reference_length': 64119}
{'bleu': 0.198371742684942, 'precisions': [0.5315606244388085, 0.2568820991022271, 0.142397411588707, 0.0829066886870355], 'brevity_penalty': 0.9900000837512514, 'length_ratio': 0.9900497512437811, 'translation_length': 63481, 'reference_length': 64119}


In [27]:
validation_translations = generate_translations(val_data)
val_refs = [example["translation"]["en"] for example in val_data]
import evaluate



Translating: 100%|██████████| 2169/2169 [11:05<00:00,  3.26it/s]


NameError: name 'validation_refs' is not defined

In [28]:
bleu = evaluate.load("bleu")
results_1 = bleu.compute(predictions=validation_translations, references=val_refs, max_order=1)
print(results_1)
results_2 = bleu.compute(predictions=validation_translations, references=val_refs, max_order=2)
print(results_2)
results_3 = bleu.compute(predictions=validation_translations, references=val_refs, max_order=3)
print(results_3)
results_4 = bleu.compute(predictions=validation_translations, references=val_refs, max_order=4)
print(results_4)

{'bleu': 0.49367290122530916, 'precisions': [0.5096196569803608], 'brevity_penalty': 0.9687085151904449, 'length_ratio': 0.9691880369485175, 'translation_length': 45012, 'reference_length': 46443}
{'bleu': 0.32905018030188277, 'precisions': [0.5096196569803608, 0.22640804798916975], 'brevity_penalty': 0.9687085151904449, 'length_ratio': 0.9691880369485175, 'translation_length': 45012, 'reference_length': 46443}
{'bleu': 0.23163742460062606, 'precisions': [0.5096196569803608, 0.22640804798916975, 0.11849739404071197], 'brevity_penalty': 0.9687085151904449, 'length_ratio': 0.9691880369485175, 'translation_length': 45012, 'reference_length': 46443}
{'bleu': 0.16812812937367191, 'precisions': [0.5096196569803608, 0.22640804798916975, 0.11849739404071197, 0.0663654774887054], 'brevity_penalty': 0.9687085151904449, 'length_ratio': 0.9691880369485175, 'translation_length': 45012, 'reference_length': 46443}


In [33]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=validation_translations, references=val_refs, model_type="distilbert-base-uncased")
print(results)

results = bertscore.compute(predictions=test_translations, references=test_refs, model_type="distilbert-base-uncased")
print(results)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.7615596055984497, 0.9409469366073608, 0.9765104651451111, 0.8342380523681641, 0.8931359648704529, 0.847620964050293, 0.8680810928344727, 0.8261321783065796, 0.8516960144042969, 0.9173384308815002, 0.8862889409065247, 0.778535008430481, 0.8676823377609253, 0.8386300802230835, 0.9089933037757874, 0.9204580187797546, 0.8180539011955261, 0.8242896795272827, 0.840498685836792, 0.9005566835403442, 0.9201635122299194, 0.8513404130935669, 0.8843384981155396, 0.8449311256408691, 0.8072896003723145, 0.8719667196273804, 0.8648838996887207, 0.9142832159996033, 0.8509263396263123, 0.8809707164764404, 0.8613384962081909, 0.8456195592880249, 0.9224295616149902, 0.8686690330505371, 0.8532232046127319, 0.8777071237564087, 0.8942518830299377, 0.8677610158920288, 0.8533433079719543, 0.9132816791534424, 0.911223828792572, 0.893591046333313, 0.8809033036231995, 0.879845380783081, 0.7789468765258789, 0.9208436012268066, 0.8123300075531006, 0.7951772809028625, 0.7582777142524719, 0.886484503

In [30]:
import json

with open("validation_translations.json", "w") as f:
    json.dump(validation_translations, f)


with open("test_translations.json", "w") as f:
    json.dump(test_translations, f)


In [37]:
with open("validation_refs_true.json", "w") as f:
    json.dump(val_refs, f)


with open("test_refs_true.json", "w") as f:
    json.dump(test_refs, f)

In [31]:
import pickle

with open("validation_translations.pkl", "wb") as f:
    pickle.dump(validation_translations, f)

# Save test translations using pickle
with open("test_translations.pkl", "wb") as f:
    pickle.dump(test_translations, f)


In [32]:
pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 984.2 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=test_translations, references=test_refs)
print(results)
results = meteor.compute(predictions=validation_translations, references=val_refs)
print(results)